In [404]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.utils import shuffle, resample

In [405]:
def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` node and the value is the respective value feed to that node.

    Returns a list of sorted nodes.
    """
    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def forward_and_backward(graph):
    """
    Performs a forward pass through a list of sorted nodes.

    Arguments:

        `output_node`: A node in the graph, should be the output node (have no outgoing edges).
        `sorted_nodes`: A topologically sorted list of nodes.

    Returns the output Node's value
    """

    for n in graph:
        n.forward()
        
    for n in graph[::-1]:
        n.backward()

In [406]:
class Node(object):
    def __init__(self, inbound_nodes=[]):
        """
        Node's constructor (runs when the object is instantiated). Sets
        properties that all nodes need.
        """
        # properties
        self.inbound_nodes = inbound_nodes
        self.value = None
        self.outbound_nodes = []
        self.gradients = {}
        for node in inbound_nodes:
            node.outbound_nodes.append(self)
            
        self.value = None
    
    def forward(self):
        """
        Forward propagation.

        Compute the output value based on `inbound_nodes` and
        store the result in self.value.
        """
        raise NotImplemented
        
    def backward(self):
        """
        Every node that uses this class as a base class will
        need to define its own `backward` method.
        """
        raise NotImplementedError

In [407]:
class Input(Node):
    def __init__(self):
        # An Input node has no inbound nodes,
        # so no need to pass anything to the Node instantiator.
        Node.__init__(self)
        
    def forward(self, value=None):
        pass

    def backward(self):
        self.gradients = {self: 0}
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]
            self.gradients[self] += grad_cost * 1

In [408]:
class Add(Node):
    def __init__(self, x, y):
        Node.__init__(self, [x, y])
        
    def forward(self):
        x, y = self.inbound_nodes
        self.value = x.value + y.value

In [409]:
class Linear(Node):
    def __init__(self, X, Y, b):
        Node.__init__(self, [X, Y, b])
        
    def forward(self):
        x, y, z = self.inbound_nodes
        self.value = np.dot(x.value, y.value) + z.value
        
    def backward(self):
        """
        Calculates the gradient based on the output values.
        """
        # Initialize a partial for each of the inbound_nodes.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]
            inputs, weights, bias = self.inbound_nodes
            self.gradients[inputs] += np.dot(grad_cost, self.inbound_nodes[1].value.T)
            self.gradients[weights] += np.dot(self.inbound_nodes[0].value.T, grad_cost)
            self.gradients[bias] += np.sum(grad_cost, axis=0, keepdims=False)

In [410]:
class Sigmoid(Node):
    def __init__(self, node):
        Node.__init__(self, [node])
        
    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def forward(self):
        linear = self.inbound_nodes[0].value
        self.value = self._sigmoid(linear)
    
    def backward(self):
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]
            inputs = self.inbound_nodes[0]
            self.gradients[inputs] += self.value * (1 - self.value) * grad_cost

In [411]:
class MSE(Node):
    def __init__(self, y, a):
        Node.__init__(self, [y, a])
        
    def forward(self):
        """
        Calculates the mean squared error.
        NOTE: We reshape these to avoid possible matrix/vector broadcast
        errors.
        
        For example, if we subtract an array of shape (3,) from an array of shape
        (3,1) we get an array of shape(3,3) as the result when we want
        an array of shape (3,1) instead.
        
        Making both arrays (3,1) insures the result is (3,1) and does
        an elementwise subtraction as expected.
        """
        self.m = self.inbound_nodes[0].value.shape[0]

        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)
        self.diff = y - a
        self.value = 1/len(y) * np.sum(np.square(self.diff))
        
    def backward(self):
        self.gradients[self.inbound_nodes[0]] = (2 / self.m) * self.diff
        self.gradients[self.inbound_nodes[1]] = (-2 / self.m) * self.diff

In [432]:
def sgd_update(trainables, learning_rate=1e-2):
    """
    Updates the value of each trainable with SGD.

    Arguments:

        `trainables`: A list of `Input` Nodes representing weights/biases.
        `learning_rate`: The learning rate.
    """
    for t in trainables:
        t.value -= (learning_rate * t.gradients[t] )


In [434]:
# Load data
data = load_boston()
X_ = data['data']
y_ = data['target']

# Normalize data
X_ = (X_ - np.mean(X_, axis=0)) / np.std(X_, axis=0)

n_features = X_.shape[1]
n_hidden = 10
W1_ = np.random.randn(n_features, n_hidden)
b1_ = np.zeros(n_hidden)
W2_ = np.random.randn(n_hidden, 1)
b2_ = np.zeros(1)

# Neural network
X, y = Input(), Input()
W1, b1 = Input(), Input()
W2, b2 = Input(), Input()

l1 = Linear(X, W1, b1)
s1 = Sigmoid(l1)
l2 = Linear(s1, W2, b2)
cost = MSE(y, l2)

feed_dict = {
    X: X_,
    y: y_,
    W1: W1_,
    b1: b1_,
    W2: W2_,
    b2: b2_
}

epochs = 1000
# Total number of examples
m = X_.shape[0]
batch_size = 11
steps_per_epoch = m // batch_size

graph = topological_sort(feed_dict)
trainables = [W1, b1, W2, b2]

print("Total number of examples = {}".format(m))

# Step 4
for i in range(epochs):
    loss = 0
    for j in range(steps_per_epoch):
        # Step 1
        # Randomly sample a batch of examples
        X_batch, y_batch = resample(X_, y_, n_samples=batch_size)

        # Reset value of X and y Inputs
        X.value = X_batch
        y.value = y_batch

        # Step 2
        forward_and_backward(graph)

        # Step 3
        sgd_update(trainables)

        loss += graph[-1].value

    print("Epoch: {}, Loss: {:.3f}".format(i+1, loss/steps_per_epoch))

Total number of examples = 506
Epoch: 1, Loss: 99.645
Epoch: 2, Loss: 33.963
Epoch: 3, Loss: 29.872
Epoch: 4, Loss: 20.862
Epoch: 5, Loss: 26.198
Epoch: 6, Loss: 17.409
Epoch: 7, Loss: 14.757
Epoch: 8, Loss: 14.474
Epoch: 9, Loss: 16.627
Epoch: 10, Loss: 10.987
Epoch: 11, Loss: 15.670
Epoch: 12, Loss: 13.906
Epoch: 13, Loss: 12.232
Epoch: 14, Loss: 11.291
Epoch: 15, Loss: 12.765
Epoch: 16, Loss: 10.976
Epoch: 17, Loss: 9.596
Epoch: 18, Loss: 9.705
Epoch: 19, Loss: 11.328
Epoch: 20, Loss: 10.716
Epoch: 21, Loss: 10.786
Epoch: 22, Loss: 11.713
Epoch: 23, Loss: 8.631
Epoch: 24, Loss: 11.574
Epoch: 25, Loss: 9.300
Epoch: 26, Loss: 9.142
Epoch: 27, Loss: 10.763
Epoch: 28, Loss: 9.986
Epoch: 29, Loss: 9.185
Epoch: 30, Loss: 8.193
Epoch: 31, Loss: 7.732
Epoch: 32, Loss: 7.779
Epoch: 33, Loss: 9.617
Epoch: 34, Loss: 6.978
Epoch: 35, Loss: 8.512
Epoch: 36, Loss: 8.258
Epoch: 37, Loss: 7.267
Epoch: 38, Loss: 9.429
Epoch: 39, Loss: 8.652
Epoch: 40, Loss: 8.137
Epoch: 41, Loss: 7.778
Epoch: 42, Lo